# Acoustic Data Feature Extraction Notebook

## Import Packages

In [1]:
# Audio processing
import librosa
import librosa.display as ld
import IPython.display as ipd
# Data cleaning
import os
import pandas as pd
import numpy as np
from numpy import mean as mean
from numpy import var as var
# Data visualization
import matplotlib.pyplot as plt
# Statistics
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from scipy.stats import spearmanr
# Advanced options
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

## Define Constants

In [2]:
FRAMESIZE = 1024
HOPLENGTH = 512
# 13 mels for fingerprint and more than 20 mels for word detection
MELS = 13 
SPLITFREQ = 2000

## Read Data

In [3]:
# Cluster data for future work
opd_data = []
others_data = []
directory= "A:\Professional\Engineering CU\DSP_Data_New"
for dir in os.listdir(directory):
    if dir == "open the door":
        for filename in os.listdir(os.path.join(directory, dir)):
            path_opd = os.path.join(directory, dir, filename)
            opd_data.append(path_opd)
    else:
        for filename in os.listdir(os.path.join(directory, dir)):
            path_others = os.path.join(directory, dir, filename)
            others_data.append(path_others)

In [4]:
# Group all the data
data = []
data.extend(opd_data)
data.extend(others_data)

In [5]:
# Determine speakers
speaker = []
for d in data:
    if "adham" in d: # append 0 for adham
        speaker.append(0)
    if "mahmoud" in d or "mohmoud" in d: # append 1 for mahmoud
        speaker.append(1)
    if "ahmed" in d: # append 2 for ahmed
        speaker.append(2)
    if "maha" in d: # append 3 for maha
        speaker.append(3)
len(speaker)

88

In [6]:
# Fingerprint Dataframe
fp_df = pd.DataFrame()
fp_df['data'] = data
fp_df['speaker'] = speaker
fp_df

,data,speaker
0,A:\Professional\Engineering CU\DSP_Data_New\op...,0
1,A:\Professional\Engineering CU\DSP_Data_New\op...,0
2,A:\Professional\Engineering CU\DSP_Data_New\op...,0
3,A:\Professional\Engineering CU\DSP_Data_New\op...,0
4,A:\Professional\Engineering CU\DSP_Data_New\op...,0
...,...,...
83,A:\Professional\Engineering CU\DSP_Data_New\un...,3
84,A:\Professional\Engineering CU\DSP_Data_New\un...,3
85,A:\Professional\Engineering CU\DSP_Data_New\un...,3
86,A:\Professional\Engineering CU\DSP_Data_New\un...,1


In [20]:
# Functions we will use
def transform_audio(audio, FRAMESIZE, HOPLENGTH, MELS, SPLITFREQ):
    
    audio_array, sr = librosa.load(audio)
    ae_audio = fancy_amplitude_envelope(audio_array, FRAMESIZE, HOPLENGTH)
    rms_audio = librosa.feature.rms(audio_array, frame_length=FRAMESIZE, hop_length=HOPLENGTH)[0]
    mel_audio = librosa.feature.melspectrogram(audio_array, sr=sr, n_fft=FRAMESIZE, hop_length=HOPLENGTH, n_mels=MELS)
    log_mel_audio = librosa.power_to_db(mel_audio)[0]
    mfccs_audio = librosa.feature.mfcc(y=audio_array, n_mfcc=MELS, sr=sr, n_fft=FRAMESIZE, hop_length=HOPLENGTH)[0]
    spec_audio = librosa.stft(audio_array, n_fft=FRAMESIZE, hop_length=HOPLENGTH)
    ber_audio = band_energy_ratio(spec_audio, SPLITFREQ, sr)
    sc_audio = librosa.feature.spectral_centroid(y=audio_array, sr=sr, n_fft=FRAMESIZE, hop_length=HOPLENGTH)[0]
    delta_mfccs_audio = librosa.feature.delta(mfccs_audio)[0]
    # Added New
    chromagram = librosa.feature.chroma_stft(audio_array, sr=sr, n_fft=FRAMESIZE, hop_length=HOPLENGTH)[0]
    tone = librosa.feature.tonnetz(y=audio_array, sr=sr)[0]
    
    # Because if framesize is bigger than that the delta_mfcc will tends to zero
    if FRAMESIZE <= 1024:        
        return np.hstack((mean(log_mel_audio), var(log_mel_audio), mean(mfccs_audio), var(mfccs_audio), mean(delta_mfccs_audio), var(delta_mfccs_audio), mean(sc_audio), var(sc_audio), mean(ber_audio), var(ber_audio), mean(rms_audio), var(rms_audio), mean(ae_audio), var(ae_audio), mean(chromagram), var(chromagram), mean(tone), var(tone)))    
    return np.hstack((mean(log_mel_audio), var(log_mel_audio), mean(mfccs_audio), var(mfccs_audio), mean(sc_audio), var(sc_audio), mean(ber_audio), var(ber_audio), mean(rms_audio), var(rms_audio), mean(ae_audio), var(ae_audio), mean(chromagram), var(chromagram), mean(tone), var(tone)))    
        



def calculate_split_frequency_bin(split_frequency, sample_rate, num_frequency_bins):
    """Infer the frequency bin associated to a given split frequency."""
    
    frequency_range = sample_rate / 2
    frequency_delta_per_bin = frequency_range / num_frequency_bins
    split_frequency_bin = int(split_frequency / frequency_delta_per_bin)
    return split_frequency_bin

def fancy_amplitude_envelope(signal, framesize, hoplength):
    return np.array([max(signal[i:i+framesize]) for i in range(0, len(signal), hoplength)])

In [21]:
z = len(fp_df['data'])
features_length_small = len(transform_audio(fp_df['data'].iloc[0], FRAMESIZE, HOPLENGTH, MELS, SPLITFREQ))
features_length_big = len(transform_audio(fp_df['data'].iloc[0], FRAMESIZE*4, HOPLENGTH*4, MELS, SPLITFREQ))

In [22]:
# Small chunk
df_small_chunk = pd.DataFrame()
df_small_chunk['data'] = fp_df['data'].copy()
df_small_chunk['speaker'] = fp_df['speaker'].copy()

for i in tqdm(range(features_length_small)):
    df_small_chunk[f'feature {i}'] = np.zeros(z)

temp_array_small = []
for index, row in tqdm(df_small_chunk.iloc[:].iterrows()):
    if index < len(df_small_chunk)-1:
        array_1_small = transform_audio(df_small_chunk.loc[index, ['data']][0], FRAMESIZE, HOPLENGTH, MELS, SPLITFREQ)
        array_2_small = transform_audio(df_small_chunk.loc[index+1, ['data']][0], FRAMESIZE, HOPLENGTH, MELS, SPLITFREQ)
        combined_array_small = np.vstack((array_1_small, array_2_small))[0]
        temp_array_small.append(list(combined_array_small))
    
last_element_small = transform_audio(df_small_chunk.loc[len(df_small_chunk)-1, ['data']][0], FRAMESIZE, HOPLENGTH, MELS, SPLITFREQ)
temp_array_small.append(list(last_element_small))


df_small_chunk.iloc[:,2:] = pd.DataFrame(temp_array_small).copy()
df_small_chunk.fillna(0, inplace=True)
df_small_chunk

100%|████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 1991.75it/s]
86it [00:42,  2.01it/s]


,data,speaker,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10,feature 11,feature 12,feature 13,feature 14,feature 15,feature 16,feature 17
0,A:\Professional\Engineering CU\DSP_Data_New\op...,0,-27.946878,274.171082,-591.251709,12277.317383,8.665314,0.0,1886.831218,812302.325372,4.970767,83.972900,0.013634,0.000580,0.027843,0.002087,0.637220,0.090833,-0.014063,0.001900
1,A:\Professional\Engineering CU\DSP_Data_New\op...,0,-31.470943,339.712433,-596.309570,13379.764648,12.491157,0.0,1956.928862,638793.280123,0.000000,0.000000,0.015390,0.000874,0.033648,0.004332,0.475199,0.105591,0.004940,0.001837
2,A:\Professional\Engineering CU\DSP_Data_New\op...,0,-28.741650,302.544525,-595.602173,11657.736328,14.608976,0.0,1822.757696,569811.932642,0.000000,0.000000,0.013316,0.000455,0.023385,0.001250,0.515015,0.096196,0.003447,0.003507
3,A:\Professional\Engineering CU\DSP_Data_New\op...,0,-27.627691,345.469208,-546.374390,9220.183594,6.154559,0.0,1813.157956,754177.385738,0.000000,0.000000,0.020185,0.001819,0.051932,0.020891,0.452060,0.094325,0.002377,0.002053
4,A:\Professional\Engineering CU\DSP_Data_New\op...,0,-28.467091,310.062164,-586.068176,11761.992188,17.225109,0.0,1940.781830,782546.842008,0.000000,0.000000,0.012783,0.000393,0.023735,0.001037,0.541509,0.086454,0.021529,0.004497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,A:\Professional\Engineering CU\DSP_Data_New\ot...,1,-30.693451,314.899933,-599.285339,9289.529297,14.274533,0.0,1837.229917,635421.163529,0.000000,0.000000,0.011432,0.000396,0.031979,0.003083,0.599059,0.089354,-0.001541,0.002740
82,A:\Professional\Engineering CU\DSP_Data_New\ot...,1,-28.741058,392.246460,-593.382019,11467.471680,15.913840,0.0,1796.561642,893880.377595,0.000000,0.000000,0.016019,0.000697,0.038313,0.004353,0.615777,0.071275,0.001249,0.002031
83,A:\Professional\Engineering CU\DSP_Data_New\ot...,1,-28.945341,373.419556,-594.748413,12434.864258,9.230136,0.0,1923.029700,787767.139396,0.000000,0.000000,0.015792,0.000709,0.036189,0.003951,0.644988,0.073178,0.033660,0.001872
84,A:\Professional\Engineering CU\DSP_Data_New\ot...,1,-30.175066,373.202393,-599.534485,14154.817383,11.886560,0.0,2018.465078,794939.732887,0.000000,0.000000,0.015473,0.000734,0.037878,0.004126,0.622911,0.074374,0.021225,0.002042


In [23]:
# Big chunk
df_big_chunk = pd.DataFrame()
df_big_chunk['data'] = fp_df['data'].copy()
df_big_chunk['speaker'] = fp_df['speaker'].copy()

for i in tqdm(range(features_length_big)):
    df_big_chunk[f'feature {i}'] = np.zeros(z)

temp_array_big = []
for index, row in tqdm(df_big_chunk.iloc[:].iterrows()):
    if index < len(df_big_chunk)-1:
        array_1_big = transform_audio(df_big_chunk.loc[index, ['data']][0], FRAMESIZE*4, HOPLENGTH*4, MELS, SPLITFREQ)
        array_2_big = transform_audio(df_big_chunk.loc[index+1, ['data']][0], FRAMESIZE*4, HOPLENGTH*4, MELS, SPLITFREQ)
        combined_array_big = np.vstack((array_1_big, array_2_big))[0]
        temp_array_big.append(list(combined_array_big))
    
last_element_big = transform_audio(df_big_chunk.loc[len(df_big_chunk)-1, ['data']][0], FRAMESIZE*4, HOPLENGTH*4, MELS, SPLITFREQ)
temp_array_big.append(list(last_element_big))


df_big_chunk.iloc[:,2:] = pd.DataFrame(temp_array_big).copy()
df_big_chunk

100%|████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 1784.38it/s]
86it [00:45,  1.91it/s]


,data,speaker,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10,feature 11,feature 12,feature 13,feature 14,feature 15
0,A:\Professional\Engineering CU\DSP_Data_New\op...,0,-14.460496,260.195160,-441.285034,14687.514648,2022.772114,1.146488e+06,0.003405,0.000019,0.015319,0.000505,0.043756,0.003342,0.524418,0.084185,-0.014063,0.001900
1,A:\Professional\Engineering CU\DSP_Data_New\op...,0,-18.254000,352.801788,-447.589722,16826.134766,1993.755346,5.818932e+05,0.008597,0.000353,0.017842,0.000755,0.065452,0.010407,0.346460,0.051185,0.004940,0.001837
2,A:\Professional\Engineering CU\DSP_Data_New\op...,0,-15.296220,281.437500,-443.781036,13295.675781,1841.433549,4.840813e+05,0.004390,0.000041,0.014829,0.000391,0.036475,0.002278,0.421904,0.087743,0.003447,0.003507
3,A:\Professional\Engineering CU\DSP_Data_New\op...,0,-14.086230,360.848267,-434.511139,17937.117188,1802.323680,6.514805e+05,0.009576,0.000546,0.023781,0.001585,0.110395,0.061947,0.356140,0.058979,0.002377,0.002053
4,A:\Professional\Engineering CU\DSP_Data_New\op...,0,-15.670943,299.866943,-431.716034,14096.180664,2029.046813,6.972431e+05,0.003431,0.000025,0.014061,0.000339,0.036283,0.001656,0.545310,0.116371,0.021529,0.004497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,A:\Professional\Engineering CU\DSP_Data_New\ot...,1,-16.995220,304.729370,-449.251953,11354.384766,1857.053856,4.886113e+05,0.004326,0.000032,0.012764,0.000345,0.049066,0.005321,0.575017,0.116434,-0.001541,0.002740
82,A:\Professional\Engineering CU\DSP_Data_New\ot...,1,-15.343622,396.103546,-444.714478,15264.509766,1855.142955,7.158562e+05,0.015457,0.000710,0.017563,0.000613,0.059188,0.007990,0.483364,0.069587,0.001249,0.002031
83,A:\Professional\Engineering CU\DSP_Data_New\ot...,1,-15.661103,372.149872,-451.198761,15868.918945,1933.219201,6.465387e+05,0.007466,0.000247,0.017353,0.000624,0.055144,0.006576,0.654336,0.086815,0.033660,0.001872
84,A:\Professional\Engineering CU\DSP_Data_New\ot...,1,-16.851711,371.462097,-454.140320,17565.685547,2077.205869,5.840794e+05,0.007093,0.000127,0.016585,0.000665,0.053774,0.006342,0.624366,0.063098,0.021225,0.002042


In [24]:
# A function to select the best features by checking their correlation. 
def manualFeatureSelection(features, outcome, r=0.76, pro=0.01, corrType='p'): 
    # p for pearson's correlation & s for spearman's correlation.  
    # First filter
    # Check correlation between all features and the outcome.
    x1 = []
    for i in features:
        if corrType == 's':
            rho, p = spearmanr(features[str(i)], outcome)
        else:
            rho, p = pearsonr(features[str(i)], outcome)

        if (abs(rho) > r) and (p < pro):
            x1.append(i)
    x2 = features[x1]
    # -----------------------------------------------------------------------------
    # Second filter
    # Check correlation between all features and each other to eliminate redundant features.

    corrColumns = set()  # Set of all the names of correlated columns

    if corrType == 's':
        corr_matrix = x2.corr(method='spearman')
    else:
        corr_matrix = x2.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > r:  # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                corrColumns.add(colname)
    return list(corrColumns)

In [29]:
best_features_small = manualFeatureSelection(df_small_chunk.iloc[:,2:], df_small_chunk.iloc[:,1], 0.3, 0.01, corrType='s')
best_features_small

['feature 3', 'feature 17', 'feature 11', 'feature 7']

In [30]:
best_features_big = manualFeatureSelection(df_big_chunk.iloc[:,2:], df_big_chunk.iloc[:,1], 0.3, 0.01, corrType='s')
best_features_big

['feature 3', 'feature 15', 'feature 9']

In [31]:
def stat_analysis(df, n):
    # Perform PCA to make dimensionality reduction 
    pca = PCA()

    # Fit the standardized data to the pca
    pca=pca.fit(df)
    two_first_comp_var_exp = pca.explained_variance_ratio_.cumsum()[n]
    print("The cumulative variance of the first 14 principal components is {}".format(
    round(two_first_comp_var_exp, 5)))

In [32]:
stat_analysis(df_big_chunk[best_features_big].iloc[:,:], 2)

The cumulative variance of the first 14 principal components is 1.0


In [33]:
stat_analysis(df_small_chunk[best_features_small].iloc[:,:], 2)

The cumulative variance of the first 14 principal components is 1.0


In [35]:
df_big_chunk[best_features_big].to_csv('A:\\Professional\\Engineering CU\\Open-sesame\data\\big_chunk.csv', index=False)
df_small_chunk[best_features_small].to_csv('A:\\Professional\\Engineering CU\\Open-sesame\data\\small_chunk.csv', index=False)
fp_df.iloc[:,1].to_csv('A:\\Professional\\Engineering CU\\Open-sesame\\data\\target_data.csv', index=False)

After exporting our data lets go for our model building notebook.